**SENTIMENT ANALYSIS USING GEMINI TO GET THE RATING FOR SPECIFIC CATEGORIES**

In [1]:
%pip install requests
%pip install -q -U google-generativeai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


**To run this code we need a gemini api key which can be generated using google ai studio which is free to use**

In [2]:
import google.generativeai as genai

GOOGLE_API_KEY=''#INSERT API KEY HERE
genai.configure(api_key=GOOGLE_API_KEY)

In [3]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
# %pip install pandas openpyxl requests
import pandas as pd
import time
import requests
from requests.exceptions import HTTPError
import time

model = genai.GenerativeModel('gemini-pro')

def find_review_column(df):#COLUMN FOR FINDING THE REVIEWS COLUMN
    for column in df.columns:
        if df[column].dtype == object and df[column].str.contains("review", case=False, na=False).any():
            return column
    raise ValueError("No review column found in the DataFrame")

# Function to generate content with retries
def generate_content_with_retry(prompt, model, max_retries=5, wait_time=10):
    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            return response.text
        except HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
                wait_time *= 2  
            else:
                raise e
    raise Exception("Max retries exceeded")

#ENTER NAME OF PROCESSOR TO GET THE ANALYSIS 
df = pd.read_csv('i7_g11.csv')
review_column = 4

# Add new columns for ratings
df['value_for_money'] = None
df['speed'] = None
df['heating_issues'] = None

# Loop through each row in the DataFrame
for index, row in df.iterrows():
    review = row.get(review_column)
    if not review or not isinstance(review, str) or review.strip() == "":
        print(f"Empty or invalid review at index {index}")
        continue

    prompt = f"Read the review {review} and give me a response in string which contains only 3 integers on a scale of 1-10 which represents Value for money, speed and heating issues, if these categories are mentioned in the review rate it from one to 10 and don't give commas only three integers separated by whitespace, if the review does not have the categories mentioned and is a positive review return 10 10 10 else if it is a negative review return 0 0 0"
    try:
        response_text = generate_content_with_retry(prompt, model)
        ratings = response_text.split()  # Splitting the response into a list of strings

        if len(ratings) != 3:
            print(f"Unexpected response for review at index {index}: {response_text}")
            # Handle the incomplete or incorrect response (e.g., retry, use default values, etc.)
            continue

        # Convert ratings to integers
        ratings = [int(rating) for rating in ratings]

        # Update the DataFrame with the new ratings
        df.at[index, 'value_for_money'] = ratings[0]
        df.at[index, 'speed'] = ratings[1]
        df.at[index, 'heating_issues'] = ratings[2]

        # Debug output to check if DataFrame is being updated
        print(f"Updated row {index}: value_for_money={ratings[0]}, speed={ratings[1]}, heating_issues={ratings[2]}")
    except Exception as e:
        print(f"Failed to get response for review at index {index}: {e}")
        time.sleep(61)

# Save the updated DataFrame back to a CSV file
output_file = 'i7_gen11_updated.csv'
df.to_csv(output_file, index=False)

print(f"Updated DataFrame saved to {output_file}")

C:\Users\hp\AppData\Local\Temp\ipykernel_6164\4109773434.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
C:\Users\hp\AppData\Local\Temp\ipykernel_6164\4109773434.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  review = row.get(review_column)


Updated row 0: value_for_money=10, speed=10, heating_issues=10
Updated row 1: value_for_money=10, speed=10, heating_issues=10
Updated row 2: value_for_money=10, speed=10, heating_issues=10
Updated row 3: value_for_money=10, speed=10, heating_issues=10
Updated row 4: value_for_money=10, speed=10, heating_issues=7
Updated row 5: value_for_money=10, speed=10, heating_issues=10
Updated row 6: value_for_money=10, speed=10, heating_issues=10
Updated row 7: value_for_money=10, speed=10, heating_issues=10
Updated row 8: value_for_money=10, speed=10, heating_issues=10
Updated row 9: value_for_money=10, speed=10, heating_issues=10
Updated row 10: value_for_money=10, speed=10, heating_issues=10
Updated row 11: value_for_money=10, speed=10, heating_issues=10
Updated row 12: value_for_money=0, speed=0, heating_issues=0
Updated row 13: value_for_money=10, speed=10, heating_issues=10
Updated row 14: value_for_money=10, speed=10, heating_issues=10
Updated row 15: value_for_money=10, speed=10, heating_

**CALCULATING THE AVERAGE RATING FOR VALUE FOR MONEY, SPEED AND HEATING ISSUES**

In [4]:
df_updated = pd.read_csv('i7_gen11_updated.csv')

# Calculate the average for each new column
average_value_for_money = df_updated['value_for_money'].mean()
average_speed = df_updated['speed'].mean()
average_heating_issues = df_updated['heating_issues'].mean()

# Print the averages
print(f"Average Rating for Value for Money: {average_value_for_money}")
print(f"Average Rating for Speed: {average_speed}")
print(f"Average Rating for Heating Issues: {average_heating_issues}")

Average Rating for Value for Money: 7.8359375
Average Rating for Speed: 8.0859375
Average Rating for Heating Issues: 7.484375
